In [12]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:90%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:10pt;}
div.text_cell_render.rendered_html{font-size:11pt;}
div.output {font-size:10pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:10pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:10pt;padding:5px;}
table.dataframe{font-size:10px;}
</style>
"""))

<b><font size="6" color="red">Ch 04. Tensorflow version 1의 객체 흐름</font><b>

# 1. tensorflow v2.xx에서 v1 사용하기

In [4]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() #tensorflow v2 비활성화하고 v1 활성화
import numpy as np
import pandas as pd

## Tensorflow
- 데이터 흐름 그래프(tensor 객체의 흐름)를 사용하는 수치 계산 라이브러리
- 그래프는 node(연산)와 edge로 구성
- sess = tf.Session()을 이용해서 실행 환경
- sess.run()을 통해서 실행 결과를 확인

In [7]:
# 1단계 : tensor(상수 node 1개) 정의
node1 = tf.constant('Hello, Tensorflow!')
# 2단계 : 세션(연산을 실행하는 환경) 생성
sess = tf.Session()
# 3단계 : 실제 실행
print(sess.run(node1)) #print(node1) 로는 출력되지 않음
print(sess.run(node1).decode())

b'Hello, Tensorflow!'
Hello, Tensorflow!


In [9]:
# 간단한 연산 tensor 그래프
# 1단계 : 그래프 정의
node1 = tf.constant(10, dtype=tf.float16)
node2 = tf.constant(20, dtype=tf.float16)
node3 = tf.add(node1, node2)
# 2단계 : 세션 생성
sess = tf.Session()
# 3단계 : 세션 실행 & 결과
print(sess.run([node1, node2, node3]))

[10.0, 20.0, 30.0]


In [10]:
# 타입변경
import numpy as np
node1 = tf.constant(np.array([1, 2, 3]), dtype=tf.int16)
node2 = tf.cast(node1, dtype=tf.float16)
sess = tf.Session()
print(sess.run([node1, node2]))

[array([1, 2, 3], dtype=int16), array([1., 2., 3.], dtype=float16)]


In [14]:
# 평균값 계산 : tf.reduce_mean()
data = np.array([1.,2.,3.,4.]) #정수로 넣으면
m = tf.reduce_mean(data)
sess = tf.Session()
print(sess.run(m)) #정수 2로 나온다.

2.5


In [16]:
# tf.random_normal([size]) : 평균이 0 이고 표준편차가 1인 난수 size개 발생
w = tf.random_normal([3])
sess = tf.Session()
sess.run(w)

array([-0.99864846,  0.2161388 , -1.1570383 ], dtype=float32)

In [18]:
# 변수 노드
w = tf.Variable(tf.random_normal([1]))
sess = tf.Session()
sess.run(tf.global_variables_initializer()) #변수 초기화
sess.run(w)

array([0.8381642], dtype=float32)

# 2. Tensorflow v1을 이용한 회귀분석 구현
## 2.1. 독립변수 x가 1개, 타깃(종속)변수 y가 1개

In [20]:
# tensor 그래프 정의
# dataset 확보
x = np.array([1,2,3])
y = np.array([2,3,4])
# weight와 bias
W = tf.Variable(tf.random.normal([1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')
# 예측값 Hat, Hypothesis
H = W*x + b
# cost function(손실함수 : mse)
cost = tf.reduce_mean(tf.square(H-y))
'''
학습 목적 : cost가 최소가 되는 W, b를 찾는 것
cost 함수는 2차함수이므로 곡선 그래프 - 미분값 0이 되는 방향(경사하강법 GradientDescent)
'''
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01) #learning_rate : 조정 비율
train = optimizer.minimize(cost)
# 세션 생성
sess = tf.Session()
# W, b 초기화
sess.run(tf.global_variables_initializer())
# 6000번 학습 (v2에서의 fit함수)
for step in range(6001):
    _, cost_val, W_val, b_val = sess.run([train, cost, W, b])
    if step%300==0:
        print("{}번째 cost:{}, W:{}, b:{}".format(step, cost_val, W_val, b_val))

0번째 cost:1.6108025312423706, W:[1.3152093], b:[-0.7626355]
300번째 cost:0.08631619065999985, W:[1.3404056], b:[0.22617799]
600번째 cost:0.020367128774523735, W:[1.1653544], b:[0.62411064]
900번째 cost:0.004805831238627434, W:[1.080322], b:[0.81740904]
1200번째 cost:0.0011339791817590594, W:[1.0390168], b:[0.9113053]
1500번째 cost:0.0002675762225408107, W:[1.0189527], b:[0.9569157]
1800번째 cost:6.313535413937643e-05, W:[1.0092063], b:[0.9790718]
2100번째 cost:1.4898420886311214e-05, W:[1.0044723], b:[0.98983365]
2400번째 cost:3.515882099236478e-06, W:[1.0021726], b:[0.99506146]
2700번째 cost:8.301151979139831e-07, W:[1.0010557], b:[0.9976004]
3000번째 cost:1.9632879855180363e-07, W:[1.0005136], b:[0.998833]
3300번째 cost:4.660375907405978e-08, W:[1.0002505], b:[0.99943155]
3600번째 cost:1.1083105988518582e-08, W:[1.0001221], b:[0.9997227]
3900번째 cost:2.6709869871410774e-09, W:[1.0000601], b:[0.99986404]
4200번째 cost:6.336715041577179e-10, W:[1.0000291], b:[0.99993366]
4500번째 cost:1.693211543196682e-10, W:[1.00

In [22]:
# 최종적으로 나온 회귀식 H = W*x + b
W_, b_ = sess.run([W, b])

In [23]:
def predict(x):
    return W_[0]*x + b_[0]

In [24]:
predict(5)

6.000024616718292

## 2.2. predict를 위한 placeholder 이용
- placeholder : 외부에서 데이터를 입력받을 수 있도록 하는 노드

In [ ]:
x = tf.placeholder(dtype=float32)
H = 1*x + 1
sess = tf.Session()
sess.run( [x, H], feed_dict = {x : np.array([40, 50])} )

In [25]:
# tensor 그래프 정의
# dataset 확보
x_data = np.array([1,2,3])
y_data = np.array([2,3,4])
# placeholder로 설정(x, y)
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)
# weight와 bias
W = tf.Variable(tf.random.normal([1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')
# 예측값 Hat, Hypothesis
H = W*x + b
# cost function(손실함수 : mse)
cost = tf.reduce_mean(tf.square(H-y))
'''
학습 목적 : cost가 최소가 되는 W, b를 찾는 것
cost 함수는 2차함수이므로 곡선 그래프 - 미분값 0이 되는 방향(경사하강법 GradientDescent)
'''
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01) #learning_rate : 조정 비율
train = optimizer.minimize(cost)
# 세션 생성
sess = tf.Session()
# W, b 초기화
sess.run(tf.global_variables_initializer())
# 6000번 학습 (v2에서의 fit함수)
for step in range(6001):
    _, cost_val, W_val, b_val = sess.run([train, cost, W, b], feed_dict={x:x_data, y:y_data})
    if step%300==0:
        print("{}번째 cost:{}, W:{}, b:{}".format(step, cost_val, W_val, b_val))

0번째 cost:47.156158447265625, W:[-1.3641493], b:[-0.06379046]
300번째 cost:1.3521617802325636e-05, W:[1.0042607], b:[0.99031466]
600번째 cost:3.1911702080833493e-06, W:[1.00207], b:[0.9952948]
900번째 cost:7.53528638597345e-07, W:[1.001006], b:[0.9977135]
1200번째 cost:1.7832780940807424e-07, W:[1.0004895], b:[0.9988879]
1500번째 cost:4.235981165834346e-08, W:[1.0002385], b:[0.99945796]
1800번째 cost:1.0050338339340215e-08, W:[1.0001165], b:[0.9997358]
2100번째 cost:2.451732372676929e-09, W:[1.0000576], b:[0.99986994]
2400번째 cost:5.618403520202264e-10, W:[1.0000275], b:[0.99993736]
2700번째 cost:1.6026054094897546e-10, W:[1.0000148], b:[0.999967]
3000번째 cost:5.195962063386794e-11, W:[1.0000086], b:[0.9999813]
3300번째 cost:5.195962063386794e-11, W:[1.0000086], b:[0.9999813]
3600번째 cost:5.195962063386794e-11, W:[1.0000086], b:[0.9999813]
3900번째 cost:5.195962063386794e-11, W:[1.0000086], b:[0.9999813]
4200번째 cost:5.195962063386794e-11, W:[1.0000086], b:[0.9999813]
4500번째 cost:5.195962063386794e-11, W:[1.00

In [26]:
# 예측하기 (predict 별도 함수 없음)
sess.run(H, feed_dict={x:5})

array([6.0000243], dtype=float32)

## 2.3. scale이 다른 데이터들의 회귀분석 구현(scale 조정X)

In [27]:
x_data = np.array([1,2,5,8,10])
y_data = np.array([5,15,68,80,95])
# placeholder 설정
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)
# Weight & Bias
W = tf.Variable(tf.random_normal([1]))
b = tf.Variable(tf.random.normal([1]))
# Hypothesis
H = W*x + b
# cost (손실함수)
cost = tf.reduce_mean(tf.square(H-y))
# 경사하강법
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)
# Session & 변수 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# 학습
for step in range(1, 6001):
    _, cost_val = sess.run([train, cost], feed_dict={x:x_data, y:y_data})
    if step%300 == 0:
        print("{}/6000 번째 cost: {}".format(step+1, cost_val))

# local minima에 갇힌 걸까

301/6000 번째 cost: 79.14790344238281
601/6000 번째 cost: 79.13969421386719
901/6000 번째 cost: 79.13946533203125
1201/6000 번째 cost: 79.13944244384766
1501/6000 번째 cost: 79.13945007324219
1801/6000 번째 cost: 79.13946533203125
2101/6000 번째 cost: 79.13946533203125
2401/6000 번째 cost: 79.13946533203125
2701/6000 번째 cost: 79.13946533203125
3001/6000 번째 cost: 79.13946533203125
3301/6000 번째 cost: 79.13946533203125
3601/6000 번째 cost: 79.13946533203125
3901/6000 번째 cost: 79.13946533203125
4201/6000 번째 cost: 79.13946533203125
4501/6000 번째 cost: 79.13946533203125
4801/6000 번째 cost: 79.13946533203125
5101/6000 번째 cost: 79.13946533203125
5401/6000 번째 cost: 79.13946533203125
5701/6000 번째 cost: 79.13946533203125
6001/6000 번째 cost: 79.13946533203125


## 2.4. scale이 다른 데이터들의 회귀분석 구현(scale 조정O)
### scale 조정 방법: 모든 데이터를 일정범위 내로 조정
- normalization(정규화) : 모든 데이터를 0~1 사이로 조정
```
                       X - Xmin
    normalization = ────────
                      Xmax - Xmin
```
        - 위의 식을 써도 되지만 라이브러리를 추천(sklearn.preprocessing.MinMaxScaler)
- standardization(표준화) : 데이터의 평균을 0, 표준편차를 1로 조정
```
                        X - Xmean
    standardization = ────────
                       Xstd(표준편차)
```
        - 위의 식을 써도 되지만 라이브러리를 추천(sklearn.preprocessing.StandardScaler)

In [33]:
# 라이브러리 쓰지 않고 정규화
x_data = np.array([1, 2, 5, 8, 10])
y_data = np.array([5, 15, 68, 80, 95])
norm_scaled_x_data = (x_data - x_data.min())/(x_data.max() - x_data.min())
norm_scaled_y_data = (y_data - y_data.min())/(y_data.max() - y_data.min())
norm_scaled_x_data, norm_scaled_y_data

(array([0.        , 0.11111111, 0.44444444, 0.77777778, 1.        ]),
 array([0.        , 0.11111111, 0.7       , 0.83333333, 1.        ]))

In [38]:
# 라이브러리 사용해 정규화
x_data = np.array([1, 2, 5, 8, 10]).reshape(-1,1) #(5,1)
y_data = np.array([5, 15, 68, 80, 95]).reshape(-1,1)
from sklearn.preprocessing import MinMaxScaler
scaler_x = MinMaxScaler() # 독립변수 x를 정규화시킬 객체
scaler_x.fit(x_data)
norm_scaled_x_data = scaler_x.transform(x_data)
scaler_y = MinMaxScaler() # 타깃변수 x를 정규화시킬 객체
norm_scaled_y_data = scaler_y.fit_transform(y_data)
norm_scaled_y_data

array([[0.        ],
       [0.11111111],
       [0.7       ],
       [0.83333333],
       [1.        ]])

In [41]:
# 라이브러리 쓰지 않고 표준화
x_data = np.array([1, 2, 5, 8, 10])
y_data = np.array([5, 15, 68, 80, 95])
std_scaled_x_data = (x_data - x_data.mean())/x_data.std()
std_scaled_y_data = (y_data - y_data.mean())/y_data.std()
np.column_stack([x_data, norm_scaled_x_data, std_scaled_x_data])

array([[ 1.        ,  0.        , -1.22474487],
       [ 2.        ,  0.11111111, -0.93313895],
       [ 5.        ,  0.44444444, -0.05832118],
       [ 8.        ,  0.77777778,  0.81649658],
       [10.        ,  1.        ,  1.39970842]])

In [42]:
np.column_stack([y_data, norm_scaled_y_data, std_scaled_y_data])

array([[ 5.        ,  0.        , -1.32373476],
       [15.        ,  0.11111111, -1.04563922],
       [68.        ,  0.7       ,  0.42826713],
       [80.        ,  0.83333333,  0.76198177],
       [95.        ,  1.        ,  1.17912508]])

In [43]:
# 라이브러리 사용해 표준화
x_data = np.array([1, 2, 5, 8, 10]).reshape(-1,1) #(5,1)
y_data = np.array([5, 15, 68, 80, 95]).reshape(-1,1)
from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
stan_scaled_x_data = scaler_x.fit_transform(x_data)
scaler_y = StandardScaler()
stan_scaled_y_data = scaler_y.fit_transform(y_data)

In [44]:
np.column_stack([y_data, norm_scaled_y_data, stan_scaled_y_data])

array([[ 5.        ,  0.        , -1.32373476],
       [15.        ,  0.11111111, -1.04563922],
       [68.        ,  0.7       ,  0.42826713],
       [80.        ,  0.83333333,  0.76198177],
       [95.        ,  1.        ,  1.17912508]])

In [45]:
# scale 조정된 데이터를 다시 복구 : inverse_transform() 이용
scaler_y.inverse_transform(stan_scaled_y_data)

array([[ 5.],
       [15.],
       [68.],
       [80.],
       [95.]])

In [47]:
x_data = np.array([1,2,5,8,10])
y_data = np.array([5,15,68,80,95])
# placeholder 설정
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)
# Weight & Bias
W = tf.Variable(tf.random_normal([1]))
b = tf.Variable(tf.random.normal([1]))
# Hypothesis
H = W*x + b
# cost (손실함수)
cost = tf.reduce_mean(tf.square(H-y))
# 경사하강법
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)
# Session & 변수 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# 학습
for step in range(1, 6001):
    _, cost_val = sess.run([train, cost], feed_dict={x:norm_scaled_x_data, y:norm_scaled_y_data})
    if step%300 == 0:
        print("{}/6000 번째 cost: {}".format(step, cost_val))

300/6000 번째 cost: 0.01045567262917757
600/6000 번째 cost: 0.009939023293554783
900/6000 번째 cost: 0.009811936877667904
1200/6000 번째 cost: 0.009780574589967728
1500/6000 번째 cost: 0.009772837162017822
1800/6000 번째 cost: 0.00977092795073986
2100/6000 번째 cost: 0.009770453907549381
2400/6000 번째 cost: 0.009770339354872704
2700/6000 번째 cost: 0.009770313277840614
3000/6000 번째 cost: 0.009770306758582592
3300/6000 번째 cost: 0.009770303964614868
3600/6000 번째 cost: 0.009770301170647144
3900/6000 번째 cost: 0.009770303964614868
4200/6000 번째 cost: 0.009770305827260017
4500/6000 번째 cost: 0.009770305827260017
4800/6000 번째 cost: 0.009770305827260017
5100/6000 번째 cost: 0.009770305827260017
5400/6000 번째 cost: 0.009770305827260017
5700/6000 번째 cost: 0.009770305827260017
6000/6000 번째 cost: 0.009770305827260017
